In [1]:
using RigidBodyDynamics
using ForwardDiff
using Plots
using MeshCatMechanisms

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/20bf9867c6d8c22b253c6453b69609af00fc1eeb-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/20bf9867c6d8c22b253c6453b69609af00fc1eeb-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

INFO: Loading HttpServer methods...


In [2]:
urdf = "doublependulum.urdf"
doublependulum = parse_urdf(Float64,urdf)

Spanning tree:
Vertex: world (root)
  Vertex: base_link, Edge: base_link_to_world
    Vertex: upper_link, Edge: shoulder
      Vertex: lower_link, Edge: elbow
No non-tree joints.

In [3]:
state = MechanismState(doublependulum)
fixedjoint, shoulder, elbow = joints(doublependulum)

3-element Array{RigidBodyDynamics.Joint{Float64,JT} where JT<:RigidBodyDynamics.JointType{Float64},1}:
 base_link_to_world
 shoulder          
 elbow             

In [4]:
function fc(x,u)
    state = MechanismState{eltype(x)}(doublependulum)
    
    # set the state variables:
    set_configuration!(state, x[1:2])
    set_velocity!(state, x[3:4])
    
    # return momentum converted to an `Array` (as this is the format that ForwardDiff expects)
    [x[3];x[4]; Array(mass_matrix(state))\[0.; u] - Array(mass_matrix(state))\Array(dynamics_bias(state))]
end

function f(x,u,dt)
    return x + fc(x + fc(x,u)*dt/2,u)*dt
end

f (generic function with 1 method)

In [5]:
function fc2(S)
    state = MechanismState{eltype(S)}(doublependulum)
    
    # set the state variables:
    set_configuration!(state, S[1:2])
    set_velocity!(state, S[3:4])
    [S[3];S[4]; Array(mass_matrix(state))\[0.;S[5]] - Array(mass_matrix(state))\Array(dynamics_bias(state)); 0.;0.]
end

function f2(S)
    return S + fc2(S + fc2(S)*S[end]/2)*S[end]
end

Df = S-> ForwardDiff.jacobian(f2, S)

(::#1) (generic function with 1 method)

In [6]:
#iLQR
function rollout(x0,U,f,dt,tf)
    N = convert(Int64,floor(tf/dt))
    X = zeros(size(x0,1),N)
    X[:,1] = x0
    for k = 1:N-1
      X[:,k+1] = f(X[:,k],U[:,k],dt)
    end
    return X
end

function cost(X,U,Q,R,Qf,xf)
    N = size(X,2)
    J = 0.
    for k = 1:N-1
      J += 0.5*(X[:,k] - xf)'*Q*(X[:,k] - xf) + 0.5*U[:,k]'*R*U[:,k]
    end
    J += 0.5*(X[:,N] - xf)'*Qf*(X[:,N] - xf)
    return J
end

function backwardpass(X,U,Df,Q,R,Qf,xf)
    n, N = size(X)
    m = size(U,1)
    S = zeros(n,n,N)
    s = zeros(n,N)
    K = zeros(m,n,N-1)
    l = zeros(m,N-1)
    C1 = zeros(1,n,N)
    C2 = zeros(1,m,N)
    C3 = zeros(n,n,N)
    C4 = zeros(m,m,N)
    C5 = zeros(m,n,N)
    S[:,:,N] = Qf
    s[:,N] = Qf*(X[:,N] - xf)
    v1 = 0
    v2 = 0

    mu = 0.
    k = N-1
    while k >= 1
        q = Q*(X[:,k] - xf)
        r = R*(U[:,k])
        Df_aug = Df([X[:,k];U[:,k];dt])
        A = Df_aug[1:size(X,1),1:size(X,1)]
        B = Df_aug[1:size(X,1),size(X,1)+1:size(X,1)+size(U,1)]
        C1[:,:,k] = q' + s[:,k+1]'*A
        C2[:,:,k] = r' + s[:,k+1]'*B
        C3[:,:,k] = Q + A'*S[:,:,k+1]*A
        C4[:,:,k] = R + B'*(S[:,:,k+1] + mu*eye(n))*B
        C5[:,:,k] = B'*(S[:,:,k+1] + mu*eye(n))*A

        # regularization
        if any(x->x < 0, eigvals(C4[:,:,k]))
            mu_reg = mu_reg + 1;
            k = N-1;
            println("REG.!!!")
        end

        K[:,:,k] = C4[:,:,k]\C5[:,:,k]
        l[:,k] = C4[:,:,k]\C2[:,:,k]'
        s[:,k] = C1[:,:,k] - C2[:,:,k]*K[:,:,k] + l[:,k]'*C4[:,:,k]*K[:,:,k] - l[:,k]'*C5[:,:,k]
        S[:,:,k] = C3[:,:,k] + K[:,:,k]'*C4[:,:,k]*K[:,:,k] - K[:,:,k]'*C5[:,:,k] - C5[:,:,k]'*K[:,:,k]

        # terms for line search
        v1 += l[:,k]'*C2[:,:,k]'
        v2 += l[:,k]'*C4[:,:,k]*l[:,k]
        
        k = k - 1;
    end
    return K, l, S, s, v1, v2
end

function forwardpass(x0,U,f,Q,R,Qf,xf,dt,tf)
    X = rollout(x0,U,f,dt,tf)
    J = cost(X,U,Q,R,Qf,xf)
    return X, J
end

function solve(x0,m,f,Df,Q,R,Qf,xf,dt,tf,iterations=100,eps=1e-5,c1=0.25,c2=0.75)
    N = convert(Int64,floor(tf/dt))
    U = zeros(m,N-1)
    X = zeros(size(x0,1),N)
    X, J = forwardpass(x0,U,f,Q,R,Qf,xf,dt,tf)
    println("Initial Cost: $J")
    
    for i = 1:iterations
      println("*** Iteration: $i ***")
      K, l, S, s, v1, v2 = backwardpass(X,U,Df,Q,R,Qf,xf)
        
      ### check for NaN
      if any(isnan,K)
          println("K")
      end
      if any(isnan,l)
          println("l")
      end
        
      println("K: $K")
      println("l: $l")
        
      ##
      if any(isnan,S)
          println("S")
      end
      if any(isnan,s)
          println("s")
      end
        
      println("S: $S")
      println("s: $s")
    
      X_prev = copy(X)
      J_prev = copy(J)
      U_ = zeros(m,N-1)
      J = Inf
      alpha = 1
      dV = 0
      z = 0
      while J > J_prev || z < c1 || z > c2
          X[:,1] = x0
          for k = 1:N-1
              U_[:,k] = U[:,k] - K[:,:,k]*(X[:,k] - X_prev[:,k]) - alpha*l[:,k]
              X[:,k+1] = f(X[:,k],U_[:,k],dt);
                
              if any(isnan,X) || any(isnan,U_)
                  println("*****X problem*******")
                  println("bad u: $(U_[:,k])")
                  println("K: $(K[:,:,k]), l: $(l[:,k])")
                  println("dx: $(X[:,k] - X_prev[:,k])")
                  println("X: $(X[:,k])")
                  println("X_prev: $(X_prev[:,k])")

                  break
              end
          end
          println("rollout: $X")
          println("new control: $U_")
          if any(isnan(X))
              J = Inf
          else
              J = cost(X,U_,Q,R,Qf,xf)
          end
          dV = alpha*v1 + (alpha^2)*v2/2
          z = (J_prev - J)/dV[1,1]
          alpha = alpha/2;
      end

      U = copy(U_);
      println("New cost: $J")
      println("-----")
      println("Expected improvement: $(-dV)")
      println("Actual improvement: $(J - J_prev)")
      println("(z = $z)")

      if abs(J - J_prev) < eps
          println("eps criteria met at iteration: $i")
          println("-----SOLVED-----")
          break
      end

    end
    
    return X, U
end

solve (generic function with 5 methods)

In [7]:
n = 4
m = 1

# initial and goal states
x0 = [0.;0.;0.;0.]
xf = [pi;0.;0.;0.]

set_configuration!(state, x0[1:2])
set_velocity!(state, x0[3:4])

# costs
Q = 1e-5*eye(n)
Qf = 25.*eye(n)
R = 1e-5*eye(m)

# simulation
tf = 1.
dt = 0.01

0.01

In [ ]:
X, U = @time solve(x0,m,f,Df,Q,R,Qf,xf,dt,tf)

P = plot(linspace(0,tf,size(X,2)),X[1,:],title="Acrobat (iLQR and Rigid Body Dynamics)")
P = plot!(linspace(0,tf,size(X,2)),X[2,:],ylabel="State")

Initial Cost: 123.37494046779551

In [ ]:
W = plot(linspace(0,tf,size(X,2)),[U[1,1] U[1,:]']',title="Double Pendulum (iLQR and Rigid Body Dynamics)",ylabel="Control")

In [ ]:
vis = MechanismVisualizer(doublependulum,URDFVisuals(urdf));
IJuliaCell(vis)

In [ ]:
for i = 1:size(X,2)
    set_configuration!(state, X[1:2,i])
    set_configuration!(vis, configuration(state))
    sleep(0.01)
end